In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import initializers, regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sns
from numpy import fft
import tensorflow as tf

# FPGheArt - Competition

### Lorenzo Buriola - 2021860
### Filippo Conforto - 2021856
### Lorenzo Domenichetti - 2011653




# Data loading

In [6]:
x = np.loadtxt("DATA/x_ts_comp_N10000.csv", delimiter =",")
categ_y = np.loadtxt("DATA/y_ts_comp_N10000.csv", delimiter =",")

scaler = StandardScaler()
x =  scaler.fit_transform(x.T).T

In [7]:
y = to_categorical(categ_y)

In [8]:
L = len(x[0])
input_shape = (len(x[0]),1)
N_categ = 3

# Augmentation

In [17]:
# Data augmentation obtained by flipping the time series

temp = np.flip(x, axis = 1)

x2 = np.concatenate([x,temp], axis = 0)
y2 = np.concatenate([y,y], axis = 0)

L = len(x2[0])
input_shape = (len(x2[0]),1)
N_categ = 3

x2 = x2.reshape(x2.shape[0],L,1)


# Model definition

In [70]:
ini = initializers.RandomNormal(mean = 0, stddev = 0.05)

model = Sequential()

model.add(Conv1D(filters = 10, kernel_size = 21,
                    kernel_regularizer = regularizers.l2(0.01), #L2 regularizer with intermediate value of lambda
                    kernel_initializer=ini,
                    padding = "same", #Data padding
                    activation = "relu",
                    input_shape = input_shape
                    ))

model.add(MaxPooling1D(5))

model.add(Flatten())
model.add(Dense(3,activation="relu")) #Small dense layer 

model.add(Dense(N_categ, activation="softmax"))

# compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer = "adam", metrics = ["accuracy"])

In [71]:
model.summary()

model.save_weights("competition_weights.h5") #Da togliere


Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 60, 10)            220       
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 12, 10)            0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 120)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 3)                 363       
_________________________________________________________________
dense_37 (Dense)             (None, 3)                 12        
Total params: 595
Trainable params: 595
Non-trainable params: 0
_________________________________________________________________


# Final Fit

In [ ]:
model.fit(x2, y2, batch_size = 250, epochs = 150, 
                verbose = 0, shuffle = True, use_multiprocessing=True,workers=16)
model.save_weights("FPGheArt_parameters.h5")

In [ ]:
model.load_weights("FPGheArt_parameters.h5")


x_test = np.loadtxt("DATA/x_test_N10000.csv", delimiter =",") #Test set import

x_test =  scaler.fit_transform(x.T).T                         #Test set scaling

x_test = x_test.reshape(x_test.shape[0],L,1)                  #Test set reshaping

np.savetxt("FPGheArt_yhat.h5", np.argmax(model.predict(x_test), axis=1)) #Save prediction results